In [41]:
import pandas as pd
import os
import random
import PIL.Image
import numpy as np
import scipy.stats as ss
import tensorflow as tf
import tensorflow_addons as tfa
from matplotlib import pyplot as plt

In [2]:
import tensorflow_addons as tfa
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
def load_image(path, mode='RGB'):
    return PIL.Image.open(path)


def to_array(image):
    return np.asarray(image)


def to_image(array, mode='RGB'):
    return PIL.Image.fromarray(np.uint8(array), mode=mode)


def resize(image, size):
    return tf.image.resize(image, size)


def resize_smallest_side_different_scales(image, smallest_side_to=(224, 384)):
    height, width = to_array(image).shape[:2]
    scaled_list = []

    if height < width:

        for scale in smallest_side_to:
            scaled = tf.image.resize(image, (scale, width))
            scaled_list.append(scaled)

        return scaled_list

    else:

        for scale in smallest_side_to:
            scaled = tf.image.resize(image, (height, scale))
            scaled_list.append(scaled)

        return scaled_list


def resize_with_aspect_ratio(image, target_width=(128, 256, 512), input_shape=(224, 224)):
    h, w = to_array(image).shape[:2]
    r = h / w
    resized = []

    for width in target_width:
        resized_h = int(r * width)
        resized_img = resize(image, (resized_h, width))
        resized.append(
            to_image(resize(tf.image.resize_with_crop_or_pad(resized_img, input_shape[0], input_shape[1]), (128, 128))))

    return resized


def bounding_boxes(offsets, dim):
    boxes = []

    for i in offsets:
        offset_height, offset_width = i
        target_height, target_width = dim
        boxes.append([offset_height, offset_width, target_height, target_width])

    return boxes


def random_sectioning(image, offsets, dims):
    boxes = bounding_boxes(offsets, dims)
    image_sections = []
    height, width = to_array(image).shape[:2]

    if (height < height // 2 + dims[0]) and (width < width // 2 + dims[1]):
        image = tf.image.resize(image, (dims[0] * 2, dims[1] * 2))

    if (height > height // 2 + dims[0]) and (width < width // 2 + dims[1]):
        image = tf.image.resize(image, (height, dims[1] * 2))

    if (height < height // 2 + dims[0]) and (width > width // 2 + dims[1]):
        image = tf.image.resize(image, (dims[0] * 2, width))

    for box in boxes:
        if random.choice([True, False]):
            section = tf.image.crop_to_bounding_box(image, box[0], box[1], box[2], box[3])
            image_sections.append(resize(section, (128, 128)))

    return image_sections


def aggressive_cropping(image, copies, crop_window, resize_smallest_side=None, output_shape=(128, 128)):
    global resized_copies

    if resize_smallest_side is not None:
        if isinstance(resize_smallest_side, int):
            img = resize(to_array(image), (resize_smallest_side, resize_smallest_side))

        if isinstance(resize_smallest_side, (list, tuple)):
            resized_copies = [tf.image.resize(to_array(image), (size, size)) for size in resize_smallest_side]

    if isinstance(crop_window, int):
        crops = [tf.image.random_crop(to_array(image), (crop_window, crop_window)) for _ in range(copies)]

        return [resize(crop_img, output_shape) for crop_img in crops]

    elif isinstance(crop_window, (list, tuple)):
        crops = [tf.image.random_crop(to_array(image), crop_window) for _ in range(copies)]

        return [resize(crop_img, output_shape) for crop_img in crops]

## Pipeline

In [4]:
def pipeline(file_name, src, dst, label):
    processed = []
    image = load_image(os.path.join(src, file_name))
    height, width = to_array(image).shape[:2]

    sections = random_sectioning(to_array(image),
                                 [[0, 0], [height // 2, 0], [0, width // 2], [height // 2, width // 2],
                                  [height // 4, width // 4]],
                                 [224, 224])
    resize_small_side = resize_smallest_side_different_scales(to_array(image), (224, 384))
    resized_with_aspect_ratio = resize_with_aspect_ratio(to_array(image))
    resized_original = tf.image.resize_with_pad(to_array(image), 224, 224)

    for i, arr in enumerate(sections):
        filename = f'r-sec-{i}-{file_name}'
        processed.append([filename, label])
        to_image(arr).save(os.path.join(dst, filename))

    for i, arr in enumerate(resized_with_aspect_ratio):
        filename = f'r-to-ar-{i}-{file_name}'
        processed.append([filename, label])
        to_image(arr).save(os.path.join(dst, filename))

    for j, img in enumerate(resize_small_side):
        rand_crop = aggressive_cropping(to_image(img), 2, (224, 224, 3))

        for i, arr in enumerate(rand_crop):
            filename = f'agr-crop-{j}-{i}-{file_name}'
            processed.append([filename, label])
            to_image(arr).save(os.path.join(dst, filename))

    to_image(resized_original).save(os.path.join(dst, file_name))

    return processed

In [5]:
train_meta = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv')
train_meta

In [6]:
src = '../input/sorghum-id-fgvc-9/train_images/'
dst = 'train'
meta = []

if not os.path.isdir(dst):
    os.mkdir(dst)

for i in range(train_meta.shape[0]):
    file, label = train_meta.iloc[i]

    if os.path.exists(os.path.join(src, file)):
        temp = pipeline(file, src, dst, label)
        meta.append(temp)

    print(f'{i + 1}/{train_meta.shape[0]}', end='\r')

In [7]:
meta[0]

In [ ]:
# df1 = pd.DataFrame(empty, columns=['a_files', 'label'])
# df2 = pd.DataFrame(os.listdir('train'), columns=['a_files'])
# final = df2.merge(df1, how='inner', on='a_files')
# final

In [12]:
empty = []

for i in range(len(meta)):
    empty += meta[i][:11]

len(empty)

In [13]:
final = pd.DataFrame(empty, columns=['a_files', 'label'])
final

In [22]:
figure, axes = plt.subplots(nrows=1, ncols=6, figsize=[18, 6], dpi=300)

for i in range(len(axes)):
    axes[i].imshow(load_image(os.path.join('train', random.choice(final['a_files'].values))))

In [8]:
def inception(x, filters, projection, classes=None, aux=False, name=None, aux_name=None):
    f_1x1, f_3x3, f_3x3_reduce, f_5x5, f_5x5_reduce = filters
    x1 = Conv2D(filters=f_1x1, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x3_reducer = Conv2D(filters=f_3x3_reduce, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x5_reducer = Conv2D(filters=f_5x5_reduce, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    x3 = Conv2D(filters=f_3x3, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x3_reducer)
    x5 = Conv2D(filters=f_5x5, kernel_size=(5, 5), strides=(1, 1), activation=relu, padding='same')(x5_reducer)
    proj = Conv2D(filters=projection, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(pool)

    x = concatenate([x1, x3, x5, proj], axis=3, name=name)

    return x


def model_builder(shape, classes):
    input_layer = Input(shape=shape)
    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), activation=relu, padding='same')(input_layer)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [64, 128, 96, 32, 16], projection=32, name='inception_3a')
    x = inception(x, [128, 192, 128, 96, 32], projection=64, name='inception_3b')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [192, 208, 96, 48, 16], projection=64, name='inception_4a')

    aux_1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='valid')(aux_1)
    aux_1 = Dense(units=1024, activation=relu)(aux_1)
    aux_1 = Dropout(rate=0.7)(aux_1)
    aux_1 = Flatten()(aux_1)
    aux_out1 = Dense(units=classes, activation=softmax, name='aux_out1')(aux_1)

    x = inception(x, [160, 224, 112, 64, 24], projection=64, name='inception_4b')
    x = inception(x, [128, 256, 128, 64, 24], projection=64, name='inception_4c')
    x = inception(x, [112, 288, 144, 64, 32], projection=64, name='inception_4d')
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_4e')

    aux_2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='valid')(aux_2)
    aux_2 = Dense(units=1024, activation=relu)(aux_2)
    aux_2 = Dropout(rate=0.7)(aux_2)
    aux_2 = Flatten()(aux_2)
    aux_out2 = Dense(units=classes, activation=softmax, name='aux_out2')(aux_2)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_5a')
    x = inception(x, [384, 384, 192, 128, 48], projection=128, name='inception_5b')
    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)
    x = Dropout(rate=0.4)(x)
    x = Flatten()(x)
    output_layer = Dense(units=classes, activation=softmax, name='main_out')(x)

    model = Model(input_layer, [output_layer, aux_out1, aux_out2])
    model.compile(optimizer=Adam(learning_rate=0.001), loss=categorical_crossentropy,
                  loss_weights={'main_out': 1, 'aux_out1': 0.3, 'aux_out2': 0.3},
                  metrics=['accuracy', tfa.metrics.F1Score(num_classes=classes, threshold=0.5)])
    model.summary()

    return model

In [9]:
model = model_builder((224, 224, 3), 100)

In [10]:
generator = ImageDataGenerator(rescale=1 / 255.,
                               horizontal_flip=True,
                               vertical_flip=True,
                               brightness_range=(0.3, 0.6),
                               validation_split=0.3)

In [23]:
train_batches = generator.flow_from_dataframe(dataframe=final,
                                              directory='train',
                                              x_col='a_files',
                                              y_col='label',
                                              batch_size=32,
                                              target_size=(224, 224),
                                              subset='training')

validation_batches = generator.flow_from_dataframe(dataframe=final,
                                                   directory='train',
                                                   x_col='a_files',
                                                   y_col='label',
                                                   batch_size=32,
                                                   target_size=(224, 224),
                                                   subset='validation')

In [24]:
next(train_batches)[0].shape

In [25]:
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=5)

In [26]:
history = model.fit(x=train_batches,
                    validation_data=validation_batches,
                    epochs=100,
                    steps_per_epoch=2048,
                    validation_steps=1024,
                    callbacks=[early_stop, reduce_lr])

In [27]:
temp = pd.DataFrame(history.history)
temp.to_pickle('history.pkl')

In [28]:
model.save('model.hdf5')

In [47]:
train_batches.class_indices

In [ ]:
test_preds = []

for i, file in enumerate(os.listdir('../input/sorghum-id-fgvc-9/test')):
    img = resize(to_array(load_image(os.path.join('../input/sorghum-id-fgvc-9/test', file))) / 255., (224, 224))
    img_arr = np.expand_dims(to_array(img), axis=0)
    preds = (
    np.argmax(model.predict(img_arr)[0]), np.argmax(model.predict(img_arr)[1]), np.argmax(model.predict(img_arr)[2]))
    top = ss.mode(preds)[0][0]

    label = list(train_batches.class_indices.keys())[list(train_batches.class_indices.values()).index(top)]

    test_preds.append([file, label])

    print(f'{i + 1}/{len(os.listdir("../input/sorghum-id-fgvc-9/test"))}', end='\r')

In [ ]:
test_preds = pd.DataFrame(test_preds, columns=['filename', 'cultivar'])
test_preds.to_csv('submission_1.csv', index=False)